In [0]:
import tensorflow as tf
import numpy as np
from IPython.display import Image

In [0]:
# MLP Architecture

Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

mnist데이터 불러오기

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


train 데이터 -> train, validation데이터로 나누기

In [0]:
x_val = x_train[50000:60000]
x_train = x_train[0:50000]
y_val = y_train[50000:60000]
y_train = y_train[0:50000]

In [0]:

Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/reshape_mnist.png", width=500, height=250)

In [0]:
# 행렬을 array로 변환
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

print(x_train.shape)
print(x_test.shape)

(50000, 784)
(10000, 784)


In [0]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [0]:
# 정규화 (0~255까지의 숫자를 0~1로 변환, 수학에서 표준정규분포 z라 비슷하다고 보면 됨)
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

In [0]:
# y데이터를 one-hot encoding(10개의 노드가 있으면 그 값만 1로 바꾸고 나머지는 0)
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [0]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [0]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

In [0]:
x = tf.placeholder(tf.float32, [None, 784]) #784개의 input을 받음
y = tf.placeholder(tf.float32, [None, 10]) #10개의 숫자로 구분할 거야

In [0]:
def mlp(x):
  # hidden layer1
  w1 = tf.Variable(tf.random_uniform([784,256])) #784개의 input, 256개의 노드
  b1 = tf.Variable(tf.zeros([256])) #256개의 바이어스(bias), bias갯수=노드갯수
  h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
  # hidden layer2
  w2 = tf.Variable(tf.random_uniform([256,128]))
  b2 = tf.Variable(tf.zeros([128]))
  h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
  # output layer
  w3 = tf.Variable(tf.random_uniform([128,10]))
  b3 = tf.Variable(tf.zeros([10]))
  logits = tf.matmul(h2, w3) + b3

  return logits

In [0]:
logits = mlp(x) #ouput을 받고

In [0]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=y)) #logits value를 softmax에 받아서 prediction함
    #prediction한 결과를 cross entropy를 minimize

In [0]:
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)
#loss function을 minimize, 최적화


In [0]:
# 조기 종료(early stopping)
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/early_stop.png", width=500, height=250)

In [0]:
# 조기종료를 안 했을 때
# initialize
init = tf.global_variables_initializer()

# train hyperparameters
epoch_cnt = 30 # 돌리는 횟수
batch_size = 1000
iteration = len(x_train) // batch_size

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], 
                               feed_dict={x: x_train[start: end], y: y_train[start: end]})
            start += batch_size; end += batch_size
            # Compute average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        cur_val_acc = accuracy.eval({x: x_val, y: y_val})
        print("epoch: "+str(epoch)+", validation accuracy: " 
              + str(cur_val_acc) +', loss: '+str(avg_loss))
    
    # Test model
    preds = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test}))

epoch: 0, validation accuracy: 0.1162, loss: 9782.427485351558
epoch: 1, validation accuracy: 0.7973, loss: 278.3137446212769
epoch: 2, validation accuracy: 0.875, loss: 12.431630783081053
epoch: 3, validation accuracy: 0.8876, loss: 8.002866516113281
epoch: 4, validation accuracy: 0.8945, loss: 6.276257386207581
epoch: 5, validation accuracy: 0.902, loss: 5.045669846534729
epoch: 6, validation accuracy: 0.9017, loss: 4.183398394584655
epoch: 7, validation accuracy: 0.8983, loss: 3.7552951526641842
epoch: 8, validation accuracy: 0.8915, loss: 3.501420512199401
epoch: 9, validation accuracy: 0.9076, loss: 3.539449725151061
epoch: 10, validation accuracy: 0.8844, loss: 3.220013766288758
epoch: 11, validation accuracy: 0.902, loss: 3.2388172984123242
epoch: 12, validation accuracy: 0.8985, loss: 3.2839224553108215
epoch: 13, validation accuracy: 0.8914, loss: 3.1763450026512148
epoch: 14, validation accuracy: 0.8989, loss: 3.3856840467453004
epoch: 15, validation accuracy: 0.9129, loss: 2

In [0]:
# 조기종료를 했을 때
# initialize
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# train hyperparameters
epoch_cnt = 300
batch_size = 1000
iteration = len(x_train) // batch_size

earlystop_threshold = 5 # 성능이 떨어질 때, 5번정도 기다린다는 의미 떨어졌다 다시 올라가는 경우도 있어서
earlystop_cnt = 0

In [0]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    prev_train_acc = 0.0
    max_val_acc = 0.0
    
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], 
                               feed_dict={x: x_train[start: end], y: y_train[start: end]})
            start += batch_size; end += batch_size
            # Compute train average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        # train accuracy
        cur_train_acc = accuracy.eval({x: x_train, y: y_train})
        # validation accuarcy
        cur_val_acc = accuracy.eval({x: x_val, y: y_val})
        # validation loss
        cur_val_loss = loss_op.eval({x: x_val, y: y_val})
        
        print("epoch: "+str(epoch)+
              ", train acc: " + str(cur_train_acc) +
              ", val acc: " + str(cur_val_acc) )
              #', train loss: '+str(avg_loss)+
              #', val loss: '+str(cur_val_loss))
        
        if cur_val_acc < max_val_acc:
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                if earlystop_cnt == earlystop_threshold:
                    print("early stopped on "+str(epoch))
                    break
                else:
                    print("overfitting warning: "+str(earlystop_cnt))
                    earlystop_cnt += 1
            else:
                earlystop_cnt = 0
        else:
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # Save the variables to file.
            save_path = saver.save(sess, "model/model.ckpt")
        prev_train_acc = cur_train_acc

epoch: 0, train acc: 0.17326, val acc: 0.1807
epoch: 1, train acc: 0.75264, val acc: 0.7783
epoch: 2, train acc: 0.85696, val acc: 0.8699
epoch: 3, train acc: 0.8728, val acc: 0.8815
epoch: 4, train acc: 0.88684, val acc: 0.8937
epoch: 5, train acc: 0.89436, val acc: 0.8985
epoch: 6, train acc: 0.89746, val acc: 0.9008
epoch: 7, train acc: 0.89718, val acc: 0.8993
epoch: 8, train acc: 0.89722, val acc: 0.8993
overfitting warning: 0
epoch: 9, train acc: 0.88582, val acc: 0.8866
epoch: 10, train acc: 0.88132, val acc: 0.8803
epoch: 11, train acc: 0.89528, val acc: 0.8926
overfitting warning: 0
epoch: 12, train acc: 0.91422, val acc: 0.9132
epoch: 13, train acc: 0.91452, val acc: 0.9113
overfitting warning: 0
epoch: 14, train acc: 0.90752, val acc: 0.9035
epoch: 15, train acc: 0.90648, val acc: 0.9059
epoch: 16, train acc: 0.92338, val acc: 0.9233
epoch: 17, train acc: 0.90312, val acc: 0.9052
epoch: 18, train acc: 0.87682, val acc: 0.8862
epoch: 19, train acc: 0.91748, val acc: 0.9164
ov

In [0]:

# Start testing
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model/model.ckpt")
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test}))

INFO:tensorflow:Restoring parameters from model/model.ckpt
[Test Accuracy] : 0.9654
